<a href="https://colab.research.google.com/github/pritul2/ML_practice/blob/master/Malaria_cell_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

--2020-03-02 10:37:56--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  34.1MB/s    in 11s     

2020-03-02 10:38:08 (31.8 MB/s) - ‘cell_images.zip’ saved [353452851]



In [3]:
!unzip /content/cell_images.zip

Streaming output truncated to the last 5000 lines.
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_118.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_126.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_134.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_141.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_168.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_175.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_183.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_221.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_222.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_87.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_102428_cell_91.png  
 extracting: cell_images/Uninfected/C236ThinF_IMG_20151127_1

In [0]:
import os
infected = os.listdir('/content/cell_images/Parasitized/') 
uninfected = os.listdir('/content/cell_images/Uninfected/')

In [86]:
import matplotlib.pyplot as plt
import cv2
IMAGE_SIZE = 64
dataset = []
label = []
for i in infected:
  try:
    img = cv2.imread("/content/cell_images/Parasitized/"+i)
    dataset.append(cv2.resize(img,(IMAGE_SIZE,IMAGE_SIZE)))
    label.append(1)
    print(len(dataset))
  except:
    break
for i in uninfected:
  try:
    img = cv2.imread("/content/cell_images/Uninfected/"+i)
    dataset.append(cv2.resize(img,(IMAGE_SIZE,IMAGE_SIZE)))
    label.append(0)
  except:
    break

Streaming output truncated to the last 5000 lines.
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725

In [87]:
print(len(dataset))
print(len(label))
print(dataset[0].shape)

16983
16983
(64, 64, 3)


In [0]:

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,Dropout,Flatten,Dense,AveragePooling2D
import os
import tensorflow as tf
os.environ['KERAS_BACKEND']='tensorflow'

keras.backend.get_session().run(tf.local_variables_initializer())
model = None
model = Sequential()

for i in range(2): #2 CNN layers#
	model.add(Conv2D(32,(3,3),padding = "same",data_format="channels_last",activation="relu",input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)))
	model.add(MaxPooling2D(pool_size=(2,2),strides=None,padding="same",data_format="channels_last"))
	model.add(BatchNormalization()) #Using all default values#
	model.add(Dropout(rate=0.1)) #20% probability#

#Adding Dense Layers#
model.add(Flatten())

model.add(Dense(512,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))

model.add(Dense(256,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.2))

model.add(Dense(2,activation="softmax"))
#model.add(BatchNormalization())
#model.add(Dropout(rate=0.2))

model.compile("adam", "categorical_crossentropy",metrics=['accuracy'])

In [137]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_63 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_84 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_117 (Dropout)        (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_85 (Batc (None, 16, 16, 32)      

In [138]:
dataset = np.array(dataset)
label = np.array(label)
print(dataset.shape)
print(label.shape)

(16983, 64, 64, 3)
(16983,)


In [139]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
trainX,testX,trainY,testY = train_test_split(dataset,to_categorical((label)),train_size=0.6)
print(trainX.shape)
print(testX.shape)

(10189, 64, 64, 3)
(6794, 64, 64, 3)


In [169]:
import numpy as np
hist = model.fit(trainX,trainY, batch_size=100, epochs=50,verbose=1)

Epoch 1/50
10189/10189 [==============================] - 2s 170us/step - loss: 0.0079 - acc: 0.9971
Epoch 2/50
10189/10189 [==============================] - 2s 164us/step - loss: 0.0057 - acc: 0.9980
Epoch 3/50
10189/10189 [==============================] - 2s 164us/step - loss: 0.0060 - acc: 0.9981
Epoch 4/50
10189/10189 [==============================] - 2s 166us/step - loss: 0.0053 - acc: 0.9983
Epoch 5/50
10189/10189 [==============================] - 2s 165us/step - loss: 0.0040 - acc: 0.9983
Epoch 6/50
10189/10189 [==============================] - 2s 167us/step - loss: 0.0047 - acc: 0.9980
Epoch 7/50
10189/10189 [==============================] - 2s 164us/step - loss: 0.0031 - acc: 0.9992
Epoch 8/50
10189/10189 [==============================] - 2s 163us/step - loss: 0.0045 - acc: 0.9981
Epoch 9/50
10189/10189 [==============================] - 2s 161us/step - loss: 0.0028 - acc: 0.9991
Epoch 10/50
10189/10189 [==============================] - 2s 169us/step - loss: 0.0019 - a

In [0]:
plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['dice_coef'], color='b')
plt.plot(hist.history['val_dice_coef'], color='r')
plt.show()

array([[[1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.]],

       ...,

       [[1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.]],

       [[1., 0.],
        [1., 0.]]], dtype=float32)

In [152]:
print(len(testY))
accuracy_score(y[],testY)

6794


ValueError: ignored